In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
df_train = df_train.set_index("id")
df_test = df_test.set_index("id")

In [ ]:
display(df_train.shape)
display(df_test.shape)

In [ ]:
df_train.describe().T

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
X = df_train.drop("claim",axis=1)
y = df_train[["claim"]]

seed = 123

X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.2,random_state=seed,stratify=y)

In [ ]:
dtrain = lgb.Dataset(X_train, label=y_train)
validate = lgb.Dataset(X_val,label=y_val)

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    "learning_rate":0.1,
    "verbose":-1
}

num_round = 1000


model = lgb.train(params,
               train_set=dtrain, 
               valid_sets=validate,
              num_boost_round=num_round,
              early_stopping_rounds = 15,
              verbose_eval = 30,

)

In [ ]:
y_pred = model.predict(df_test)

In [ ]:
submit_df = pd.DataFrame(y_pred,index=df_test.index,columns=["claim"])

In [ ]:
submit_df.to_csv("submit.csv")